## Setup

First, import the necessary libraries. Make sure the module is in your Python path.

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add the build directory to the Python path
sys.path.insert(0, str(Path.cwd() / 'build'))
# Import the steepest descent module
import steepest_descent as sd

## Example 1: Optimize H2 Molecule

Let's optimize the geometry of a hydrogen molecule.

In [2]:
# Initialize optimizer from JSON file
optimizer = sd.SteepestDescentOptimizer(
    atoms_file_path="input/atoms/H2.xyz",
    num_alpha_electrons=1,
    num_beta_electrons=1
)

print(f"Number of atoms: {optimizer.num_atoms()}")
print(f"Initial geometry: {optimizer.get_geometry()}")

Number of atoms: 2
Initial geometry: [(1, 0.0, 0.0, 0.0), (1, 1.3984, 0.0, 0.0)]


### Calculate initial energy and gradient

In [3]:
# Calculate initial energy
initial_energy = optimizer.calculate_energy()
print(f"Initial energy: {initial_energy:.8f} eV")

# Calculate initial gradient
initial_gradient = optimizer.calculate_gradient()
print(f"Initial gradient shape: {initial_gradient.shape}")
print(f"Initial gradient norm: {np.linalg.norm(initial_gradient):.8f}")
print(f"Initial gradient:\n{initial_gradient}")

Initial energy: -40.57527851 eV
Initial gradient shape: (3, 2)
Initial gradient norm: 0.62492547
Initial gradient:
[[-0.44188904  0.44188904]
 [ 0.          0.        ]
 [ 0.          0.        ]]
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************


### Run the optimization

In [4]:
# Run optimization with custom parameters
optimizer.optimize(
    gradient_tol=1e-4,
    max_iterations=50,
    output_path="output/H2_optimized_python.xyz"
)


Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, total iterations = 1
*****************
Beginning SCF solver
SCF complete, tot

In [5]:
# Get optimized geometry
optimized_geometry = optimizer.get_geometry()
print(f"\nOptimized geometry: {optimized_geometry}")

# Calculate final energy
final_energy = optimizer.calculate_energy()
print(f"Final energy: {final_energy:.8f} eV")
print(f"Energy change: {final_energy - initial_energy:.8f} eV")


Optimized geometry: [(1, 0.011732515910764104, 0.0, 0.0), (1, 1.3866674840892361, 0.0, 0.0)]
Final energy: -40.58052152 eV
Energy change: -0.00524301 eV
Beginning SCF solver
SCF complete, total iterations = 1
*****************


## Example 2: Larger Molecule (H2O)

In [6]:
# Create a new optimizer for water molecule
optimizer_h2o = sd.SteepestDescentOptimizer(
    atoms_file_path="input/atoms/H2O.xyz",
    num_alpha_electrons=5,
    num_beta_electrons=5
)

print(f"Number of atoms: {optimizer_h2o.num_atoms()}")
print(f"Initial geometry: {optimizer_h2o.get_geometry()}")

Number of atoms: 3
Initial geometry: [(8, 0.0, 0.0, 0.2219), (1, 0.0, 1.426, -0.8876), (1, 0.0, -1.426, -0.8876)]


In [7]:
# Calculate energy before optimization
initial_energy_h2o = optimizer_h2o.calculate_energy()
print(f"Initial energy: {initial_energy_h2o:.8f} eV")

Initial energy: -505.47529331 eVBeginning SCF solver

SCF complete, total iterations = 12
*****************


In [ ]:
# Run optimization
optimizer_h2o.optimize(
    gradient_tol=1e-4,
    max_iterations=30,
    output_path="output/H2O_optimized_python.xyz"
)


In [ ]:
final_energy_h2o = optimizer_h2o.calculate_energy()
print(f"\nFinal energy: {final_energy_h2o:.8f} eV")
print(f"Energy change: {final_energy_h2o - initial_energy_h2o:.8f} eV")


Final energy: -517.12550397 eVBeginning SCF solver

Energy change: -11.65021066 eV
SCF complete, total iterations = 9
*****************


## Visualization

You can visualize the molecular geometry using matplotlib.

In [ ]:
# Map atomic numbers to element symbols
atomic_symbols = {
    1: "H", 2: "He", 3: "Li", 4: "Be", 5: "B", 6: "C", 7: "N", 8: "O", 9: "F", 10: "Ne",
    11: "Na", 12: "Mg", 13: "Al", 14: "Si", 15: "P", 16: "S", 17: "Cl", 18: "Ar",
    # Add more if needed
}

def convert_to_xyz_string(file_path):
    """
    Reads your CNDO/steepest descent output XYZ-like file and converts it
    to a proper XYZ string with element symbols.
    """
    lines = open(file_path).read().splitlines()
    num_atoms = lines[0].strip()
    comment_line = lines[1].strip()
    
    xyz_lines = []
    for line in lines[2:2+int(num_atoms)]:
        parts = line.split()
        z_num = int(parts[0])
        x, y, z = parts[1:4]
        symbol = atomic_symbols.get(z_num, str(z_num))  # fallback to number
        xyz_lines.append(f"{symbol} {x} {y} {z}")
    
    xyz_string = f"{num_atoms}\n{comment_line}\n" + "\n".join(xyz_lines)
    return xyz_string


In [ ]:
H2_Optimized = convert_to_xyz_string("output/H2_optimized_python.xyz")
H2O_Optimized = convert_to_xyz_string("output/H2O_optimized_python.xyz")
H2_MP = convert_to_xyz_string("input/atoms/H2_MPOptimized.xyz")
H2O_MP = convert_to_xyz_string("input/atoms/H2O_MPOptimized.xyz")

In [ ]:
# Create viewer
view = py3Dmol.view(width=600, height=400)

# Add H2 optimized
view.addModel(H2_Optimized, "xyz")
view.setStyle({'model': 0}, {'stick': {'radius':0.1, 'color':'red'},
                             'sphere': {'radius':0.25, 'color':'red'}})

# Add H2 MP
view.addModel(H2_MP, "xyz")
view.setStyle({'model': 1}, {'stick': {'radius':0.1, 'color':'blue'},
                             'sphere': {'radius':0.25, 'color':'blue'}})

# Zoom and show
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# Create viewer
view = py3Dmol.view(width=800, height=400)

# Add H2O optimized
view.addModel(H2O_Optimized, "xyz")
view.setStyle({'model': 0}, {'stick': {'radius':0.1, 'color':'green'},
                             'sphere': {'radius':0.25, 'color':'green'}})

# Add H2O MP
view.addModel(H2O_MP, "xyz")
view.setStyle({'model': 1}, {'stick': {'radius':0.1, 'color':'orange'},
                             'sphere': {'radius':0.25, 'color':'orange'}})

# Zoom and show
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.